<a href="https://colab.research.google.com/github/carlos-alves-one/-SSDM-Coursework-2/blob/main/Regression_CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Statistics and Statistical Data Mining
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework No.2

### Load the data

> Run the code in Python 3

In [1]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


> Run the code in R

In [1]:
# Loading necessary library
library(readr)

# Loading the datasets using base R read.csv function
credit_train <- read.csv("/content/drive/MyDrive/credit_risk/creditdefault_train.csv", sep = ";")
credit_test <- read.csv("/content/drive/MyDrive/credit_risk/creditdefault_train.csv", sep = ";")

# Displaying the first few rows of the Credit Default training dataset
head(credit_train)

# Displaying the first few rows of the Credit Default test dataset
head(credit_test)

,Y.X1.X2.X3.X4.X5.X6.X7.X8.X9.X10.X11.X12.X13.X14.X15.X16.X17.X18.X19.X20.X21.X22.X23
,<chr>
1,"1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0"
2,"0,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000"
3,"0,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679"
4,"0,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800"
5,"0,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770"
6,"0,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542"


,Y.X1.X2.X3.X4.X5.X6.X7.X8.X9.X10.X11.X12.X13.X14.X15.X16.X17.X18.X19.X20.X21.X22.X23
,<chr>
1,"1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0"
2,"0,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000"
3,"0,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679"
4,"0,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800"
5,"0,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770"
6,"0,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542"
